In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import pypomp
#import seaborn as sns
from plotnine import ggplot, geom_line, aes, facet_wrap, coord_cartesian, geom_point

In [2]:
def funky_transform(lst):
    """Transform rho to perturbation scale"""
    out = [np.log((1 + x)/(1 - x)) for x in lst]
    return out

### Notes about pkl structure:

- Dictionary with two entries
    - fit_out: A list containing output of the fitting step. Each entry is a replicate from the fitting step.
        - Various entries: A tuple of length 2 containing information for a replicate.
            - Entry 0: A JAX array containing the iterated filtering estimate of the log likelihood for each iteration (plus an extra? check when the extra is added)
            - Entry 1: A JAX array containing parameters from particles. First dimension varies iteration, second varies the particle, and the third varies the parameter.
    - pf_out: A list containing the output of the particle filtering step. 
        - Various entries: A list for a different replicate from the particle filtering step.
            - Entry 0: A numpy.float32 representing the estimated log likelihood.
            - Entry 1: A numpy.float32 representing the estimated standard deviation of the LL estimate.


In [3]:
PKL_PATH = "C:/Users/ravis/OneDrive/Documents/danny_honors_thesis/src/1d_global_out.pkl"
pkl_in = pickle.load(open(PKL_PATH, "rb"))

### Final LL estimates:

In [4]:
LL_df = pd.DataFrame({
    "Method": ["IF2"] * len(pkl_in["pf_out"]) + ["IFAD"] * len(pkl_in["pf_out"]),
    "LL": [x[0] for x in pkl_in["pf_out"]] + [x[0] for x in pkl_in["pf_out"]],
    "sd": [x[1] for x in pkl_in["pf_out"]] + [x[1] for x in pkl_in["pf_out"]]
}).sort_values(by = "LL")

print("LL Estimates (Sorted):")
print(LL_df)

LL Estimates (Sorted):
   Method            LL          sd
1     IF2 -1.179329e+04    1.659993
21   IFAD -1.179329e+04    1.659993
19    IF2 -1.179175e+04    2.058151
39   IFAD -1.179175e+04    2.058151
31   IFAD -1.177343e+04    1.261184
11    IF2 -1.177343e+04    1.261184
6     IF2 -1.176260e+04    1.457299
26   IFAD -1.176260e+04    1.457299
38   IFAD -1.175826e+04    1.730717
18    IF2 -1.175826e+04    1.730717
2     IF2 -1.173694e+04    1.042691
22   IFAD -1.173694e+04    1.042691
33   IFAD -1.056911e+04    0.000000
13    IF2 -1.056911e+04    0.000000
24   IFAD -1.045166e+04    0.000977
4     IF2 -1.045166e+04    0.000977
14    IF2 -9.645732e+03    0.000977
34   IFAD -9.645732e+03    0.000977
23   IFAD -9.340219e+03    0.000000
3     IF2 -9.340219e+03    0.000000
16    IF2  4.910526e+02  236.321701
36   IFAD  4.910526e+02  236.321701
0     IF2  1.368423e+03   54.336082
20   IFAD  1.368423e+03   54.336082
10    IF2  1.928077e+03   34.020805
30   IFAD  1.928077e+03   34.020805
7    

### Plot traces

In [5]:
traces = pd.DataFrame()

# Function to process fit traces
def extract_traces(fit_out, method_name):
    traces_list = []
    for rep in range(len(fit_out)):
        for iter in range(fit_out[rep][1].shape[0]):
            for param in range(fit_out[rep][1].shape[2]):
                param_avg = float(np.mean(fit_out[rep][1][iter, :, param]))
                traces_list.append({
                    "Method": method_name,
                    "rep": str(rep),
                    "iter": iter,
                    "param_index": param,
                    "param_value": param_avg
                })
    
    for rep in range(len(fit_out)):
        for iter in range(fit_out[rep][0].size):
            traces_list.append({
                "Method": method_name,
                "rep": str(rep),
                "iter": iter,
                "param_index": -1,
                "param_value": -float(fit_out[rep][0][iter])
            })
    
    return pd.DataFrame(traces_list)


print("Structure of fit_out_ifad:")
for rep in range(len(pkl_in["fit_out_ifad"])):
    print(f"fit_out_ifad[{rep}]: Type={type(pkl_in['fit_out_ifad'][rep])}")
    print(f"fit_out_ifad[{rep}]: Content={pkl_in['fit_out_ifad'][rep]}")   
#traces_if2 = extract_traces(pkl_in["fit_out_if2"], "IF2")
#traces_ifad = extract_traces(pkl_in["fit_out_ifad"], "IFAD")

# Combine 2 traces into 1 dataframe
#traces = pd.concat([traces_if2, traces_ifad], ignore_index = True)
#traces

Structure of fit_out_ifad:
fit_out_ifad[0]: Type=<class 'tuple'>
fit_out_ifad[0]: Content=(Array([-11603.387,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan], dtype=float32), Array([[-10.303605 ,  -8.142563 ,  -8.9361925,  -5.7551336,  -0.6149848,
        -11.717516 ],
       [        nan,         nan,         nan,         nan,         nan,
                nan],
       [        nan,         nan,         nan,         nan,         nan,
                nan],
       [        nan,         nan,         nan,         nan,         nan,
                nan],
       [        nan,         nan,         nan,         nan,         nan,
                nan],
       [        nan,         nan,         nan,         nan,         nan,
                nan],
       [        nan,         nan,  

In [6]:
plot_params = (
    ggplot(
        traces.loc[traces["param_index"] != -1], 
        aes(x = "iter", y = "param_value", group = "rep", color = "rep")
    ) 
    + geom_line()
    + facet_wrap("param_index", scales = "free")
)
print(plot_params)

KeyError: 'param_index'

In [ ]:
# LL traces for IF2 and IFAD
plot_LL = (
    ggplot(
        traces.loc[traces["param_index"] == -1], 
        aes(x = "iter", y = "param_value", group = "rep", color = "rep")
    ) 
    + geom_line()
    + facet_wrap("param_index", scales = "free")
    + coord_cartesian(ylim = (11000, 11850))
)
print(plot_LL)

KeyError: 'param_index'

In [ ]:
# Weizhe values
print("mu: " + str(np.log(3.68e-4)))
print("kappa: " + str(np.log(3.14e-2)))
print("theta: " + str(np.log(1.12e-4)))
print("xi: " + str(np.log(2.27e-3)))
print("rho: " + str(funky_transform([-7.38e-1])))
print("v0: " + str(np.log(7.66e-3**2)))
# Exact is 11847.12 from profiling in Weizhe's thesis

# Evaluating the Weizhe model with the Weizhe parameters under 
# output/1d_global/weizhe_eval yielded LL 11849.65, sd 1.1217372.

mu: -7.907427619795343
kappa: -3.4609473860679296
theta: -9.09701168666918
xi: -6.087975447488826
rho: [np.float64(-1.8921458020642405)]
v0: -9.743486590459273
